In [1]:
import pandas as pd
from haversine import haversine, Unit
import folium
from tqdm import tqdm
from geopandas import GeoDataFrame, points_from_xy

In [2]:
df_map = pd.read_csv('dataset_final.csv')

In [3]:
df_map.head(2)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado,Dir2,Full_Address,Coords
0,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM VILLA COLOMBIA,TEXACO,Carrera 15 No. 51 - 40,GASOLINA EXTRA OXIGENADA,17610.0,A,Carrera 15 No. 51 - 40,"Carrera 15 No. 51 - 40, Cali, Colombia","(3.4437824, -76.5000414)"
1,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM VILLA COLOMBIA,TEXACO,Carrera 15 No. 51 - 40,GASOLINA CORRIENTE OXIGENADA,9440.0,A,Carrera 15 No. 51 - 40,"Carrera 15 No. 51 - 40, Cali, Colombia","(3.4437824, -76.5000414)"


In [4]:
df_map = df_map.drop_duplicates(subset=['Nombre_comercial','Bandera','Producto'])

In [5]:
len(df_map)

382

In [6]:
df_map.value_counts('Coords')

Coords
(3.4516467, -76.5319854)           12
(3.4232547, -76.5611128)            9
(3.4666254, -76.5208301)            4
(3.3427661, -76.5313195)            3
(3.4399635, -76.5032397)            3
                                   ..
(3.4490144, -76.52301450000002)     2
(3.3935255, -76.53609829999999)     1
(3.4298078, -76.47569639999999)     1
(3.4321869, -76.50901929999999)     1
(3.4664512, -76.48529429999999)     1
Name: count, Length: 137, dtype: int64

In [7]:
df_map[df_map['Coords'] == "('NA', 'NA')"]['Direccion'].unique()

array([], dtype=object)

In [8]:
df_map.info()

<class 'pandas.core.frame.DataFrame'>
Index: 382 entries, 0 to 952
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Periodo              382 non-null    int64  
 1   Mes                  382 non-null    int64  
 2   Codigo_departamento  382 non-null    int64  
 3   Departamento         382 non-null    object 
 4   Codigo_municipio     382 non-null    int64  
 5   Municipio            382 non-null    object 
 6   Nombre_comercial     382 non-null    object 
 7   Bandera              382 non-null    object 
 8   Direccion            382 non-null    object 
 9   Producto             382 non-null    object 
 10  Precio               382 non-null    float64
 11  Estado               382 non-null    object 
 12  Dir2                 382 non-null    object 
 13  Full_Address         382 non-null    object 
 14  Coords               382 non-null    object 
dtypes: float64(1), int64(4), object(10)
memory us

In [9]:
df_map['Coords'][0].replace('(','').replace(')','').split(',')

['3.4437824', ' -76.5000414']

In [10]:
df_map['LAT'] = df_map['Coords'].apply(lambda x: float(x.replace('(','').replace(')','').split(',')[0].strip()))
df_map['LNG'] = df_map['Coords'].apply(lambda x: float(x.replace('(','').replace(')','').split(',')[1].strip()))

In [11]:
df_map.head(3)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado,Dir2,Full_Address,Coords,LAT,LNG
0,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM VILLA COLOMBIA,TEXACO,Carrera 15 No. 51 - 40,GASOLINA EXTRA OXIGENADA,17610.0,A,Carrera 15 No. 51 - 40,"Carrera 15 No. 51 - 40, Cali, Colombia","(3.4437824, -76.5000414)",3.443782,-76.500041
1,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM VILLA COLOMBIA,TEXACO,Carrera 15 No. 51 - 40,GASOLINA CORRIENTE OXIGENADA,9440.0,A,Carrera 15 No. 51 - 40,"Carrera 15 No. 51 - 40, Cali, Colombia","(3.4437824, -76.5000414)",3.443782,-76.500041
2,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM VILLA COLOMBIA,TEXACO,Carrera 15 No. 51 - 40,BIODIESEL EXTRA,9300.0,A,Carrera 15 No. 51 - 40,"Carrera 15 No. 51 - 40, Cali, Colombia","(3.4437824, -76.5000414)",3.443782,-76.500041


## Funciones

In [12]:
def calc_dist(geo_source, point2, unit):
    if unit == 'Km':
        distance = haversine(geo_source, point2, Unit.KILOMETERS)
    elif unit == 'm':
        distance = haversine(geo_source, point2, Unit.METERS)
    elif unit == 'miles':
        distance = haversine(geo_source, point2, Unit.MILES)

    return round(distance, 2)


def distance_estac(geo_source, df, radio, unit):
    distancia = []
    source = []

    for i in tqdm(range(len(df)), colour='green'):
        distancia.append(calc_dist(geo_source, df['POINT'][i], unit))
        source.append(geo_source)

    new_df = df.copy()
    new_df['SOURCE'] = source
    new_df['DISTANCE'] = distancia
    new_df = new_df[new_df['DISTANCE'] <= radio]
    new_df.reset_index(inplace=True)
    new_df.drop(columns='index')
    return new_df.sort_values(by='DISTANCE', ascending=True)

def transform_df_map(df):
    cordenadas = []
    for i in range(len(df)):
        try:
            coord = float(df['LAT'][i]),float(df['LNG'][i])
            cordenadas.append(coord)
        except Exception as e:
            print(e)
            cordenadas.append('EMPTY')

    df['POINT'] = cordenadas
    df = df[df['POINT'] != 'EMPTY']
    df = df.reset_index()
    df = df.drop(columns = 'index')
    new_df = df.copy()
    
    return new_df

## Filtrar ciudad

In [13]:
cities = list(df_map['Municipio'].unique())
cities

['CALI']

In [14]:
df_city = df_map[df_map['Municipio'] == cities[0]]
df_city.reset_index(inplace=True)
df_city.drop(columns='index',inplace=True)
df_city

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado,Dir2,Full_Address,Coords,LAT,LNG
0,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM VILLA COLOMBIA,TEXACO,Carrera 15 No. 51 - 40,GASOLINA EXTRA OXIGENADA,17610.0,A,Carrera 15 No. 51 - 40,"Carrera 15 No. 51 - 40, Cali, Colombia","(3.4437824, -76.5000414)",3.443782,-76.500041
1,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM VILLA COLOMBIA,TEXACO,Carrera 15 No. 51 - 40,GASOLINA CORRIENTE OXIGENADA,9440.0,A,Carrera 15 No. 51 - 40,"Carrera 15 No. 51 - 40, Cali, Colombia","(3.4437824, -76.5000414)",3.443782,-76.500041
2,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM VILLA COLOMBIA,TEXACO,Carrera 15 No. 51 - 40,BIODIESEL EXTRA,9300.0,A,Carrera 15 No. 51 - 40,"Carrera 15 No. 51 - 40, Cali, Colombia","(3.4437824, -76.5000414)",3.443782,-76.500041
3,2022,6,76,VALLE DEL CAUCA,76001,CALI,SERVICENTRO ESSO SANTA MONICA,MOBIL,AVENIDA 6N No. 22-61,BIODIESEL EXTRA,10250.0,A,AVENIDA 6N No. 22-61,"AVENIDA 6N No. 22-61, Cali, Colombia","(3.4644016, -76.5299438)",3.464402,-76.529944
4,2022,6,76,VALLE DEL CAUCA,76001,CALI,SERVICENTRO ESSO SANTA MONICA,MOBIL,AVENIDA 6N No. 22-61,GASOLINA EXTRA OXIGENADA,17950.0,A,AVENIDA 6N No. 22-61,"AVENIDA 6N No. 22-61, Cali, Colombia","(3.4644016, -76.5299438)",3.464402,-76.529944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO LA CASONA,TERPEL,TRANSVERSAL 103 NO. 84-126,BIODIESEL EXTRA,9220.0,A,TRANSVERSAL 103 NO. 84-126,"TRANSVERSAL 103 NO. 84-126, Cali, Colombia","(3.4291354, -76.47723669999999)",3.429135,-76.477237
378,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO LA CASONA,TERPEL,TRANSVERSAL 103 NO. 84-126,GASOLINA EXTRA OXIGENADA,17990.0,A,TRANSVERSAL 103 NO. 84-126,"TRANSVERSAL 103 NO. 84-126, Cali, Colombia","(3.4291354, -76.47723669999999)",3.429135,-76.477237
379,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO LA CASONA,TERPEL,TRANSVERSAL 103 NO. 84-126,GASOLINA CORRIENTE OXIGENADA,9280.0,A,TRANSVERSAL 103 NO. 84-126,"TRANSVERSAL 103 NO. 84-126, Cali, Colombia","(3.4291354, -76.47723669999999)",3.429135,-76.477237
380,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACIÃ“N DE SERVICIO EL HORMIGUERO,BIOMAX,CGTO EL HORMIGUERO VDA. CAUCA SECO,GASOLINA CORRIENTE OXIGENADA,9300.0,A,CGTO EL HORMIGUERO VDA. CAUCA SECO,"CGTO EL HORMIGUERO VDA. CAUCA SECO, Cali, Colo...","(3.4981612, -76.47221429999999)",3.498161,-76.472214


In [15]:
central_location = 'IDJIJ'
r = ['dsijsij',3.422750, -76.546462]

In [16]:
geo_source = r[1],r[2]

In [17]:
radio = 2
unit = 'Km'

## Show Map

In [18]:
m = folium.Map([geo_source[0],geo_source[1]],zoom_start=15)

folium.Circle(
    radius=int(radio) * 1000,
    location=[geo_source[0],geo_source[1]],
    color='green',
    fill='green'
).add_to(m)

folium.Marker(
    location=[geo_source[0],geo_source[1]],
    icon = folium.Icon(color='black',icon_color='white', icon = 'home', prefix='glyphicon'),
    popup="<b>Centroide</b>"
).add_to(m)

m

In [19]:
df_city.head(2)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado,Dir2,Full_Address,Coords,LAT,LNG
0,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM VILLA COLOMBIA,TEXACO,Carrera 15 No. 51 - 40,GASOLINA EXTRA OXIGENADA,17610.0,A,Carrera 15 No. 51 - 40,"Carrera 15 No. 51 - 40, Cali, Colombia","(3.4437824, -76.5000414)",3.443782,-76.500041
1,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM VILLA COLOMBIA,TEXACO,Carrera 15 No. 51 - 40,GASOLINA CORRIENTE OXIGENADA,9440.0,A,Carrera 15 No. 51 - 40,"Carrera 15 No. 51 - 40, Cali, Colombia","(3.4437824, -76.5000414)",3.443782,-76.500041


In [20]:
df_city = transform_df_map(df_city)
df_city.head(2)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado,Dir2,Full_Address,Coords,LAT,LNG,POINT
0,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM VILLA COLOMBIA,TEXACO,Carrera 15 No. 51 - 40,GASOLINA EXTRA OXIGENADA,17610.0,A,Carrera 15 No. 51 - 40,"Carrera 15 No. 51 - 40, Cali, Colombia","(3.4437824, -76.5000414)",3.443782,-76.500041,"(3.4437824, -76.5000414)"
1,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM VILLA COLOMBIA,TEXACO,Carrera 15 No. 51 - 40,GASOLINA CORRIENTE OXIGENADA,9440.0,A,Carrera 15 No. 51 - 40,"Carrera 15 No. 51 - 40, Cali, Colombia","(3.4437824, -76.5000414)",3.443782,-76.500041,"(3.4437824, -76.5000414)"


In [21]:
results = distance_estac(geo_source,df_city,radio,unit)
results.reset_index(inplace=True)
results.drop(columns='index',inplace=True)
results


100%|██████████| 382/382 [00:00<00:00, 128620.38it/s]


,level_0,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,...,Precio,Estado,Dir2,Full_Address,Coords,LAT,LNG,POINT,SOURCE,DISTANCE
0,5,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO IMBANACO No 17,TEXACO,CALLE 5 No. 38D-154,...,18800.0,A,CALLE 5 No. 38D-154,"CALLE 5 No. 38D-154, Cali, Colombia","(3.4246403, -76.5467979)",3.424640,-76.546798,"(3.4246403, -76.5467979)","(3.42275, -76.546462)",0.21
1,3,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO IMBANACO No 17,TEXACO,CALLE 5 No. 38D-154,...,9300.0,A,CALLE 5 No. 38D-154,"CALLE 5 No. 38D-154, Cali, Colombia","(3.4246403, -76.5467979)",3.424640,-76.546798,"(3.4246403, -76.5467979)","(3.42275, -76.546462)",0.21
2,4,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO IMBANACO No 17,TEXACO,CALLE 5 No. 38D-154,...,9350.0,A,CALLE 5 No. 38D-154,"CALLE 5 No. 38D-154, Cali, Colombia","(3.4246403, -76.5467979)",3.424640,-76.546798,"(3.4246403, -76.5467979)","(3.42275, -76.546462)",0.21
3,21,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO EL LIDO,ESSO,CALLE 5 No. 44-24,...,17899.0,A,CALLE 5 No. 44-24,"CALLE 5 No. 44-24, Cali, Colombia","(3.4199081, -76.54840999999999)",3.419908,-76.548410,"(3.4199081, -76.54840999999999)","(3.42275, -76.546462)",0.38
4,22,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO EL LIDO,ESSO,CALLE 5 No. 44-24,...,10099.0,A,CALLE 5 No. 44-24,"CALLE 5 No. 44-24, Cali, Colombia","(3.4199081, -76.54840999999999)",3.419908,-76.548410,"(3.4199081, -76.54840999999999)","(3.42275, -76.546462)",0.38
5,20,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO EL LIDO,ESSO,CALLE 5 No. 44-24,...,9150.0,A,CALLE 5 No. 44-24,"CALLE 5 No. 44-24, Cali, Colombia","(3.4199081, -76.54840999999999)",3.419908,-76.548410,"(3.4199081, -76.54840999999999)","(3.42275, -76.546462)",0.38
6,11,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO COMBUSCOL ESTADIO,TEXACO,cl 6 nÂ° 31-00,...,18690.0,A,cl 6 nÂ° 31-00,"cl 6 nÂ° 31-00, Cali, Colombia","(3.4231456, -76.5413805)",3.423146,-76.541381,"(3.4231456, -76.5413805)","(3.42275, -76.546462)",0.57
7,12,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO COMBUSCOL ESTADIO,TEXACO,cl 6 nÂ° 31-00,...,9660.0,A,cl 6 nÂ° 31-00,"cl 6 nÂ° 31-00, Cali, Colombia","(3.4231456, -76.5413805)",3.423146,-76.541381,"(3.4231456, -76.5413805)","(3.42275, -76.546462)",0.57
8,13,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO COMBUSCOL ESTADIO,TEXACO,cl 6 nÂ° 31-00,...,9610.0,A,cl 6 nÂ° 31-00,"cl 6 nÂ° 31-00, Cali, Colombia","(3.4231456, -76.5413805)",3.423146,-76.541381,"(3.4231456, -76.5413805)","(3.42275, -76.546462)",0.57
9,24,2022,6,76,VALLE DEL CAUCA,76001,CALI,OPECOM EDS SILOE,TERPEL,CALLE 1 OESTE No. 50-57,...,9690.0,A,CALLE 1 OESTE No. 50-57,"CALLE 1 OESTE No. 50-57, Cali, Colombia","(3.4204114, -76.5534734)",3.420411,-76.553473,"(3.4204114, -76.5534734)","(3.42275, -76.546462)",0.82


In [22]:
results['Nombre_comercial'].unique()

array(['ESTACION DE SERVICIO IMBANACO No 17',
       'ESTACION DE SERVICIO EL LIDO',
       'ESTACION DE SERVICIO COMBUSCOL ESTADIO', 'OPECOM EDS SILOE',
       'ESTACION DE SERVICIO TERPEL CAÃ‘AVERALEJO',
       'ESTACIÃ“N SE SERVICIO ROOSEVELT',
       'ESTACION DE SERVICIO DISTRACOM MIRAFLORES',
       'ESTACION DE SERVICIO SUR AUTOPISTA',
       'ESTACION DE SERVICIO AUTOMOTRIZ COMBUSCOL SAN FERNANDO',
       'ESTACION DE SERVICIO EL DIAMANTE', 'EDS PASOANCHO',
       'ESTACION DE SERVICIO OPES  PASOANCHO', 'EDS TERPEL MARBELLA',
       'ESTACION DE SERVICIO OPES EL TRIANGULO',
       'ESTACION DE SERVICIO EL CAMBIO  SAS'], dtype=object)

In [23]:
results['Producto'].unique()

array(['GASOLINA EXTRA OXIGENADA', 'BIODIESEL EXTRA',
       'GASOLINA CORRIENTE OXIGENADA'], dtype=object)

In [24]:
gdf_results = GeoDataFrame(results,geometry=points_from_xy(results.LNG,results.LAT))

In [25]:
gdf_results

,level_0,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,...,Estado,Dir2,Full_Address,Coords,LAT,LNG,POINT,SOURCE,DISTANCE,geometry
0,5,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO IMBANACO No 17,TEXACO,CALLE 5 No. 38D-154,...,A,CALLE 5 No. 38D-154,"CALLE 5 No. 38D-154, Cali, Colombia","(3.4246403, -76.5467979)",3.424640,-76.546798,"(3.4246403, -76.5467979)","(3.42275, -76.546462)",0.21,POINT (-76.54680 3.42464)
1,3,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO IMBANACO No 17,TEXACO,CALLE 5 No. 38D-154,...,A,CALLE 5 No. 38D-154,"CALLE 5 No. 38D-154, Cali, Colombia","(3.4246403, -76.5467979)",3.424640,-76.546798,"(3.4246403, -76.5467979)","(3.42275, -76.546462)",0.21,POINT (-76.54680 3.42464)
2,4,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO IMBANACO No 17,TEXACO,CALLE 5 No. 38D-154,...,A,CALLE 5 No. 38D-154,"CALLE 5 No. 38D-154, Cali, Colombia","(3.4246403, -76.5467979)",3.424640,-76.546798,"(3.4246403, -76.5467979)","(3.42275, -76.546462)",0.21,POINT (-76.54680 3.42464)
3,21,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO EL LIDO,ESSO,CALLE 5 No. 44-24,...,A,CALLE 5 No. 44-24,"CALLE 5 No. 44-24, Cali, Colombia","(3.4199081, -76.54840999999999)",3.419908,-76.548410,"(3.4199081, -76.54840999999999)","(3.42275, -76.546462)",0.38,POINT (-76.54841 3.41991)
4,22,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO EL LIDO,ESSO,CALLE 5 No. 44-24,...,A,CALLE 5 No. 44-24,"CALLE 5 No. 44-24, Cali, Colombia","(3.4199081, -76.54840999999999)",3.419908,-76.548410,"(3.4199081, -76.54840999999999)","(3.42275, -76.546462)",0.38,POINT (-76.54841 3.41991)
5,20,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO EL LIDO,ESSO,CALLE 5 No. 44-24,...,A,CALLE 5 No. 44-24,"CALLE 5 No. 44-24, Cali, Colombia","(3.4199081, -76.54840999999999)",3.419908,-76.548410,"(3.4199081, -76.54840999999999)","(3.42275, -76.546462)",0.38,POINT (-76.54841 3.41991)
6,11,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO COMBUSCOL ESTADIO,TEXACO,cl 6 nÂ° 31-00,...,A,cl 6 nÂ° 31-00,"cl 6 nÂ° 31-00, Cali, Colombia","(3.4231456, -76.5413805)",3.423146,-76.541381,"(3.4231456, -76.5413805)","(3.42275, -76.546462)",0.57,POINT (-76.54138 3.42315)
7,12,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO COMBUSCOL ESTADIO,TEXACO,cl 6 nÂ° 31-00,...,A,cl 6 nÂ° 31-00,"cl 6 nÂ° 31-00, Cali, Colombia","(3.4231456, -76.5413805)",3.423146,-76.541381,"(3.4231456, -76.5413805)","(3.42275, -76.546462)",0.57,POINT (-76.54138 3.42315)
8,13,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO COMBUSCOL ESTADIO,TEXACO,cl 6 nÂ° 31-00,...,A,cl 6 nÂ° 31-00,"cl 6 nÂ° 31-00, Cali, Colombia","(3.4231456, -76.5413805)",3.423146,-76.541381,"(3.4231456, -76.5413805)","(3.42275, -76.546462)",0.57,POINT (-76.54138 3.42315)
9,24,2022,6,76,VALLE DEL CAUCA,76001,CALI,OPECOM EDS SILOE,TERPEL,CALLE 1 OESTE No. 50-57,...,A,CALLE 1 OESTE No. 50-57,"CALLE 1 OESTE No. 50-57, Cali, Colombia","(3.4204114, -76.5534734)",3.420411,-76.553473,"(3.4204114, -76.5534734)","(3.42275, -76.546462)",0.82,POINT (-76.55347 3.42041)


In [26]:
oil = list(gdf_results['Producto'].unique())
oil

['GASOLINA EXTRA OXIGENADA', 'BIODIESEL EXTRA', 'GASOLINA CORRIENTE OXIGENADA']

In [27]:
gdf_results2 = gdf_results[gdf_results['Producto'] == oil[1]]
gdf_results2.reset_index(inplace=True)
gdf_results2.drop(columns='index',inplace=True)
gdf_results2

/tmp/ipykernel_44876/1825750634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_results2.drop(columns='index',inplace=True)


,level_0,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,...,Estado,Dir2,Full_Address,Coords,LAT,LNG,POINT,SOURCE,DISTANCE,geometry
0,3,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO IMBANACO No 17,TEXACO,CALLE 5 No. 38D-154,...,A,CALLE 5 No. 38D-154,"CALLE 5 No. 38D-154, Cali, Colombia","(3.4246403, -76.5467979)",3.424640,-76.546798,"(3.4246403, -76.5467979)","(3.42275, -76.546462)",0.21,POINT (-76.54680 3.42464)
1,22,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO EL LIDO,ESSO,CALLE 5 No. 44-24,...,A,CALLE 5 No. 44-24,"CALLE 5 No. 44-24, Cali, Colombia","(3.4199081, -76.54840999999999)",3.419908,-76.548410,"(3.4199081, -76.54840999999999)","(3.42275, -76.546462)",0.38,POINT (-76.54841 3.41991)
2,13,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO COMBUSCOL ESTADIO,TEXACO,cl 6 nÂ° 31-00,...,A,cl 6 nÂ° 31-00,"cl 6 nÂ° 31-00, Cali, Colombia","(3.4231456, -76.5413805)",3.423146,-76.541381,"(3.4231456, -76.5413805)","(3.42275, -76.546462)",0.57,POINT (-76.54138 3.42315)
3,25,2022,6,76,VALLE DEL CAUCA,76001,CALI,OPECOM EDS SILOE,TERPEL,CALLE 1 OESTE No. 50-57,...,A,CALLE 1 OESTE No. 50-57,"CALLE 1 OESTE No. 50-57, Cali, Colombia","(3.4204114, -76.5534734)",3.420411,-76.553473,"(3.4204114, -76.5534734)","(3.42275, -76.546462)",0.82,POINT (-76.55347 3.42041)
4,9,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO TERPEL CAÃ‘AVERALEJO,TERPEL,CALLE 5 No. 53A-75,...,A,CALLE 5 No. 53A-75,"CALLE 5 No. 53A-75, Cali, Colombia","(3.4107803, -76.54736989999999)",3.410780,-76.547370,"(3.4107803, -76.54736989999999)","(3.42275, -76.546462)",1.33,POINT (-76.54737 3.41078)
5,43,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACIÃ“N SE SERVICIO ROOSEVELT,TERPEL,AVENIDA ROOSEVELT 28 - 21,...,A,AVENIDA ROOSEVELT 28 - 21,"AVENIDA ROOSEVELT 28 - 21, Cali, Colombia","(3.4323519, -76.53891039999999)",3.432352,-76.538910,"(3.4323519, -76.53891039999999)","(3.42275, -76.546462)",1.36,POINT (-76.53891 3.43235)
6,2,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO DISTRACOM MIRAFLORES,MOBIL,CALLE 5 No 27 - 02,...,A,CALLE 5 No 27 - 02,"CALLE 5 No 27 - 02, Cali, Colombia","(3.4341034, -76.54166769999999)",3.434103,-76.541668,"(3.4341034, -76.54166769999999)","(3.42275, -76.546462)",1.37,POINT (-76.54167 3.43410)
7,38,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO SUR AUTOPISTA,MOBIL,CARRERA 46 No. 10-15,...,A,CARRERA 46 No. 10-15,"CARRERA 46 No. 10-15, Cali, Colombia","(3.4143198, -76.5368512)",3.414320,-76.536851,"(3.4143198, -76.5368512)","(3.42275, -76.546462)",1.42,POINT (-76.53685 3.41432)
8,15,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO AUTOMOTRIZ COMBUSCOL SAN ...,TEXACO,Calle 5 Nro. 25 - 20,...,A,Calle 5 Nro. 25 - 20,"Calle 5 Nro. 25 - 20, Cali, Colombia","(3.4353119, -76.5409882)",3.435312,-76.540988,"(3.4353119, -76.5409882)","(3.42275, -76.546462)",1.52,POINT (-76.54099 3.43531)
9,7,2022,6,76,VALLE DEL CAUCA,76001,CALI,ESTACION DE SERVICIO EL DIAMANTE,TERPEL,Carrera 29 No. 39A-35,...,A,Carrera 29 No. 39A-35,"Carrera 29 No. 39A-35, Cali, Colombia","(3.4313166, -76.5355611)",3.431317,-76.535561,"(3.4313166, -76.5355611)","(3.42275, -76.546462)",1.54,POINT (-76.53556 3.43132)


In [28]:
gdf_results2['Precio'].min()

9133.0

In [29]:
gdf_results2['Precio'].max()

10099.0

In [30]:
def marker_rest(df,mapa,unit,oil,icono):
    
    df = df[df['Producto']==oil]
    df = df.reset_index()
    df = df.drop(columns = 'index')
    
    for i in range(len(df)):
        
        if df['Precio'][i]==df['Precio'].min():
        
            html =  f"""<b>MARCA:</b> {df.Bandera[i]} <br>
                    <b>NAME:</b> {df.Nombre_comercial[i]} <br>
                    <b>PRODUCTO:</b> {df.Producto[i]} <br>
                    <b>PRECIO:</b> {df.Precio[i]} <br>
                    <b>DISTANCE:</b> {round(df.DISTANCE[i],2)}<br>
                    <b>DIRECCION:</b> {df.Direccion[i]}<br>
                    <b>UNIT:</b> {unit}<br>"""
            iframe = folium.IFrame(html,figsize=(6, 3))
            popup = folium.Popup(iframe)
            



            folium.Marker(location=[float(df['LAT'][i]),float(df['LNG'][i])],
                               icon=folium.Icon(color='darkgreen', icon_color='white',
                               icon=icono, prefix='glyphicon'),
                               popup = popup).add_to(mapa)
        
        elif df['Precio'][i]==df['Precio'].max():
        
            html =  f"""<b>MARCA:</b> {df.Bandera[i]} <br>
                    <b>NAME:</b> {df.Nombre_comercial[i]} <br>
                    <b>PRODUCTO:</b> {df.Producto[i]} <br>
                    <b>PRECIO:</b> {df.Precio[i]} <br>
                    <b>DISTANCE:</b> {round(df.DISTANCE[i],2)}<br>
                    <b>DIRECCION:</b> {df.Direccion[i]}<br>
                    <b>UNIT:</b> {unit}<br>"""
            iframe = folium.IFrame(html,figsize=(6, 3))
            popup = folium.Popup(iframe)



            folium.Marker(location=[float(df['LAT'][i]),float(df['LNG'][i])],
                               icon=folium.Icon(color='darkred', icon_color='white',
                               icon=icono, prefix='glyphicon'),
                               popup =popup).add_to(mapa)
        else :
            html =  f"""<b>MARCA:</b> {df.Bandera[i]} <br>
                    <b>NAME:</b> {df.Nombre_comercial[i]} <br>
                    <b>PRODUCTO:</b> {df.Producto[i]} <br>
                    <b>PRECIO:</b> {df.Precio[i]} <br>
                    <b>DISTANCE:</b> {round(df.DISTANCE[i],2)}<br>
                    <b>DIRECCION:</b> {df.Direccion[i]}<br>
                    <b>UNIT:</b> {unit}<br>"""
            iframe = folium.IFrame(html,figsize=(6, 3))
            popup = folium.Popup(iframe)



            folium.Marker(location=[float(df['LAT'][i]),float(df['LNG'][i])],
                               icon=folium.Icon(color='orange', icon_color='white',
                               icon=icono, prefix='glyphicon'),
                               popup =popup).add_to(mapa)
                           
    return

## SHow Map

In [31]:
m = folium.Map([geo_source[0],geo_source[1]], zoom_start= 14)

folium.Circle(
    
    radius = int(radio)*1000,
    location = [geo_source[0],geo_source[1]],
    color = 'green',
    fill = 'green'

).add_to(m)

folium.Marker(

            location = [geo_source[0],geo_source[1]],
            icon =  folium.Icon(color='black',icon_color='white',
            icon = "home",prefix = 'glyphicon'),
            popup = "<b>Centroid</b>").add_to(m)

marker_rest(gdf_results2,m,unit,oil[1],'usd')

m